<a href="https://colab.research.google.com/github/sivaharitha-s/movie_recommendation_system/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
'''import difflib

words = ["apple", "appl", "appeal"]
close_matches = difflib.get_close_matches("apple", words)
print(close_matches)'''

#Measures cosine similarity between vectors. Often used to find how similar two documents are.

'''from sklearn.metrics.pairwise import cosine_similarity

A = [[1, 0, 1]]
B = [[0, 1, 1]]
similarity = cosine_similarity(A, B)
print(similarity)'''


'from sklearn.metrics.pairwise import cosine_similarity\n\nA = [[1, 0, 1]]\nB = [[0, 1, 1]]\nsimilarity = cosine_similarity(A, B)\nprint(similarity)'

In [3]:
df1=pd.read_csv('movies.csv')
df1.head(1)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron


In [29]:
selected_features = ['genres', 'keywords', 'title', 'popularity', 'revenue',
                     'runtime', 'spoken_languages', 'tagline', 'cast', 'crew', 'director']
df=df1[selected_features]

print(df.isnull().sum())
df.head(1)


genres               28
keywords            412
title                 0
popularity            0
revenue               0
runtime               2
spoken_languages      0
tagline             844
cast                 43
crew                  0
director             30
dtype: int64


,genres,keywords,title,popularity,revenue,runtime,spoken_languages,tagline,cast,crew,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Avatar,150.437577,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron


In [30]:
df= df[df['director'].notna()]
print(df.isnull().sum())
print(df.info())

genres               14
keywords            386
title                 0
popularity            0
revenue               0
runtime               2
spoken_languages      0
tagline             822
cast                 22
crew                  0
director              0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 4773 entries, 0 to 4802
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   genres            4759 non-null   object 
 1   keywords          4387 non-null   object 
 2   title             4773 non-null   object 
 3   popularity        4773 non-null   float64
 4   revenue           4773 non-null   int64  
 5   runtime           4771 non-null   float64
 6   spoken_languages  4773 non-null   object 
 7   tagline           3951 non-null   object 
 8   cast              4751 non-null   object 
 9   crew              4773 non-null   object 
 10  director          4773 non-null   object 
dtypes: fl

In [31]:
for i in selected_features:
  df[i]=df[i].fillna('')
print(df.isnull().sum())

genres              0
keywords            0
title               0
popularity          0
revenue             0
runtime             0
spoken_languages    0
tagline             0
cast                0
crew                0
director            0
dtype: int64


In [32]:
vectorizer_features=[]
for i in selected_features:
  if(df[i].dtype=='object'):
    vectorizer_features.append(i)
print(vectorizer_features)

['genres', 'keywords', 'title', 'runtime', 'spoken_languages', 'tagline', 'cast', 'crew', 'director']


In [90]:
combine_features = ['genres', 'keywords', 'title', 'runtime',
                    'spoken_languages', 'tagline', 'cast', 'crew', 'director', 'popularity', 'revenue']
df['combined_text'] = df[combine_features].astype(str).apply(lambda row: ' '.join(row), axis=1)

print(df[['combined_text']].head(10))


                                       combined_text
0  Action Adventure Fantasy Science Fiction cultu...
1  Adventure Fantasy Action ocean drug abuse exot...
2  Action Adventure Crime spy based on novel secr...
3  Action Crime Drama Thriller dc comics crime fi...
4  Action Adventure Science Fiction based on nove...
5  Fantasy Action Adventure dual identity amnesia...
6  Animation Family hostage magic horse fairy tal...
7  Action Adventure Science Fiction marvel comic ...
8  Adventure Fantasy Family witch magic broom sch...
9  Action Adventure Fantasy dc comics vigilante s...


In [91]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(df['combined_text'])
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 818295 stored elements and shape (4773, 236398)>
  Coords	Values
  (0, 190835)	0.00493295708245524
  (0, 190962)	0.005656068743600346
  (0, 203977)	0.006977034794923524
  (0, 226586)	0.006508621893689301
  (0, 204389)	0.006504819919093048
  (0, 200045)	0.013447790617339637
  (0, 198629)	0.01396136692039911
  (0, 205402)	0.010275613151619638
  (0, 228554)	0.021062783277324127
  (0, 234394)	0.007803235849427029
  (0, 199020)	0.015692871576556258
  (0, 228288)	0.013232480002306013
  (0, 192928)	0.01793795253577288
  (0, 30439)	0.01606545675499904
  (0, 210387)	0.004144887929075795
  (0, 203237)	0.0021743650128085057
  (0, 219420)	0.3167519171849562
  (0, 203302)	0.002174821419456083
  (0, 203464)	0.007371754353821485
  (0, 203501)	0.007371754353821485
  (0, 232550)	0.007371754353821485
  (0, 203332)	0.014276383200209745
  (0, 230794)	0.0034677687834489216
  (0, 235416)	0.007876227921313346
  (0, 220462)	0.0075733181682982355
  

In [92]:
similarity = cosine_similarity(feature_vectors)
print(similarity)# taking first value and find similarity with all.and so on...........

[[1.         0.56565598 0.82370997 ... 0.355445   0.08681923 0.19976776]
 [0.56565598 1.         0.57175246 ... 0.32040543 0.07436713 0.17333048]
 [0.82370997 0.57175246 1.         ... 0.37303464 0.09668598 0.20362013]
 ...
 [0.355445   0.32040543 0.37303464 ... 1.         0.05170315 0.13569425]
 [0.08681923 0.07436713 0.09668598 ... 0.05170315 1.         0.0316308 ]
 [0.19976776 0.17333048 0.20362013 ... 0.13569425 0.0316308  1.        ]]


In [93]:
print(similarity.shape)

(4773, 4773)


In [106]:
movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : spider man


In [108]:
list_of_all_titles =df1['title'].tolist()
print(list_of_all_titles[:10])

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice']


In [109]:
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Spider-Man', 'Inside Man', 'Superman']


In [110]:
close_match =find_close_match[0]
print(close_match)

Spider-Man


In [111]:
index_of_the_movie =index_of_the_movie = df[df.title == close_match].index[0]

print(index_of_the_movie)

159


In [112]:
#Finding the similarity percentage only for specifi index that is the index of iron man
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, np.float64(0.7733275783378876)), (1, np.float64(0.5907383281396287)), (2, np.float64(0.7818987162424045)), (3, np.float64(0.8147222409178347)), (4, np.float64(0.767752261290824)), (5, np.float64(0.6997097557284071)), (6, np.float64(0.6492350124349601)), (7, np.float64(0.7199651112552498)), (8, np.float64(0.5565479724824504)), (9, np.float64(0.765080699449834)), (10, np.float64(0.5052310912836726)), (11, np.float64(0.46703757796377526)), (12, np.float64(0.5160973383782375)), (13, np.float64(0.5823355077316168)), (14, np.float64(0.6699038827648525)), (15, np.float64(0.7221913539680985)), (16, np.float64(0.7709345333549648)), (17, np.float64(0.6167543876118196)), (18, np.float64(0.5277262256033864)), (19, np.float64(0.773939255406604)), (20, np.float64(0.7645430867823331)), (21, np.float64(0.46441758140179895)), (22, np.float64(0.7605337225296097)), (23, np.float64(0.5783376290562736)), (24, np.float64(0.6212972329137265)), (25, np.float64(0.682348945826052)), (26, np.float64(0.77859

In [113]:
len(similarity_score)

4773

In [114]:
# sorting the movies based on their similarity score
sorted_similar_movies = sorted(similarity_score,key=lambda x:x[1],reverse = True)
print(sorted_similar_movies)

[(159, np.float64(1.0000000000000002)), (28, np.float64(0.8268824607218158)), (412, np.float64(0.8221901968922866)), (212, np.float64(0.8201018452489695)), (30, np.float64(0.816676732824474)), (185, np.float64(0.8155434183753851)), (365, np.float64(0.8151432454020815)), (3, np.float64(0.8147222409178347)), (186, np.float64(0.813716861747617)), (171, np.float64(0.8132532230420736)), (1430, np.float64(0.8129224590730924)), (150, np.float64(0.812002713856726)), (303, np.float64(0.8119110100398492)), (298, np.float64(0.8105592286341574)), (275, np.float64(0.8103218586016183)), (165, np.float64(0.8101237233784719)), (654, np.float64(0.8100211989753433)), (529, np.float64(0.8078418735189586)), (93, np.float64(0.8067352806384203)), (505, np.float64(0.8063684059818469)), (112, np.float64(0.8060732634226248)), (1823, np.float64(0.8060578563007067)), (583, np.float64(0.8036583143371361)), (250, np.float64(0.8035024041433826)), (296, np.float64(0.8034450975860076)), (864, np.float64(0.80341517177

In [115]:
#which show the first 30 similar movie
print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movies:
    #movie=(68, 1.0000000000000002)
    #movie[0]=68
    index = movie[0]
    title_from_index = df.iloc[index]['title']
    if i < 30:
        print(i, '.', title_from_index)
        i += 1

Movies suggested for you : 

1 . Spider-Man
2 . Jurassic World
3 . The Core
4 . The Day After Tomorrow
5 . Spider-Man 2
6 . War of the Worlds
7 . Contact
8 . The Dark Knight Rises
9 . Bad Boys II
10 . Master and Commander: The Far Side of the World
11 . Cursed
12 . Men in Black II
13 . Catwoman
14 . The Wolf of Wall Street
15 . Minority Report
16 . Hulk
17 . Blade: Trinity
18 . Tears of the Sun
19 . Terminator 3: Rise of the Machines
20 . The League of Extraordinary Gentlemen
21 . Alexander
22 . Freddy vs. Jason
23 . Big Fish
24 . The Aviator
25 . End of Days
26 . Blade II
27 . xXx
28 . Blade
29 . A.I. Artificial Intelligence
